In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import plaid
from cheap.pretrained import CHEAP_pfam_shorten_2_dim_32

In [5]:
cheap_model = CHEAP_pfam_shorten_2_dim_32()

Using tanh layer at bottleneck...
Finished loading HPCT model with shorten factor 2 and 32 channel dimensions.


In [6]:
import torch
device = torch.device("cuda")

In [8]:
from plaid.diffusion import FunctionOrganismDiffusion
from plaid.denoisers import FunctionOrganismDiT
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
N, L, C = 4, 128, 32 

denoiser = FunctionOrganismDiT(input_dim=32).to(device)
diffusion = FunctionOrganismDiffusion(model=denoiser)
diffusion = diffusion.to(device)


In [9]:
from plaid.datasets import FunctionOrganismDataModule
import torch

datamodule = FunctionOrganismDataModule(
    train_shards="/data/lux70/data/pfam/compressed/j1v1wv6w/train/shard{000..160}.tar",
    val_shards="/data/lux70/data/pfam/compressed/j1v1wv6w/val/shard{000..022}.tar",
    config_file="/data/lux70/data/pfam/compressed/j1v1wv6w/config.json",
    go_metadata_fpath="/data/lux70/data/pfam/pfam2go.csv",
    organism_metadata_fpath="/data/lux70/data/pfam/organism_counts.csv",
    cache_dir="/homefs/home/lux70/cache/plaid_data_cache/j1v1wv6w",
    train_epoch_num_batches=100_000,
    val_epoch_num_batches=1_000,
    shuffle_buffer=10_000,
    shuffle_initial=10_000,
    num_workers=4,
)
datamodule.setup()

In [52]:
train_dataloader = datamodule.train_dataloader()
train_dataset = datamodule.train_ds

batch = next(iter(train_dataloader))

In [53]:
embedding, mask, go_idx, organism_idx, pfam_id, sample_id, local_path = batch

In [55]:
mask.sum(dim=1)

tensor([201, 195, 197, 168, 174, 132,  29, 120, 231, 230, 149, 105,  85, 120,
        182, 144, 143, 229, 231, 237, 143,  92,  24, 230, 232, 238, 183, 100,
         84, 177,  71,  94,  85, 167,  40,  90,  27,  85,  89, 229, 154,  82,
         71,  26,  91,  98,  84, 243, 185, 222, 232,  77,  19,  82, 197,  20,
        125, 230,  83, 213,  32, 168,  20,  16])

In [56]:
T = 200
t = torch.arange(T).long()
t = t.to(device)

In [59]:
import einops
emb = einops.repeat(embedding[0], "l c -> b l c", b=T)
mask = einops.repeat(mask[0], "l -> b l", b=T)
mask = mask.to(device)
emb = emb.to(device)

noise = torch.randn_like(emb)
noise = noise.to(device)

In [60]:
noised = diffusion.q_sample(emb,t,noise)

In [63]:
cheap_model = cheap_model.to(device)
noised_uncompress = cheap_model.decode(noised, mask) 

In [65]:
noised_uncompress.shape

torch.Size([200, 1024, 1024])